In [1]:
import lxml
import newspaper
import mitie

In [2]:
washpost = newspaper.build("https://www.washingtonpost.com/")

In [3]:
for article in washpost.articles:
    print(article.title)

None
Frank Harden, longtime D.C. morning radio host, dies at 95
Colombians vote in divisive election that could upend peace deal with FARC
Giuliani suggests Trump may pardon Manafort after Mueller’s probe
How Judge Leon blew it with U.S. v AT&T
FBI agent removed from Russia probe for anti-Trump texts says he would testify before Congress
If more states start using Ohio’s system, how many voters will be purged?
Looking to buy a home for $800,000? Here’s what you can expect to find.
Serbia tops Costa Rica, 1-0
Happy Father’s Day, Dad. Boy, did you matter.
Social Security, HUD act on Trump’s orders in move to emasculate unions
A Georgetown rowhouse built when Lincoln was in the White House
Terms of Service
 
Tear gas triggers stampede at Venezuela nightclub, resulting in 17 deaths
My dad didn’t know how to get fit. I’ll do better for my kids.
Meet the Democratic candidates for Montgomery county executive
For Pruitt, gaining Trump’s favor — and keeping it — came through fierce allegiance
M

In [7]:
article = washpost.articles[1]

In [8]:
article.download()
article.parse()

In [9]:
print(article.text)

Every morning at 6:55, they played a hymn on the radio, and at 7:25, they broadcast a good old march. If someone in the D.C. area lost a pet, they’d announce it on the air. From 1960 to 1992, Frank Harden and Jackson Weaver ran the most popular program in a gentler and more polite Washington.

Mr. Harden, who died June 15 at his home in Chevy Chase, Md., while watching a movie with his wife, was the straight man to Weaver in a 32-year morning routine that was as vital to Washington’s identity in that era as the Redskins, the Beltway and the green-and-white awnings on so many D.C. rowhouses. He was 95.

His son, Robert Harden, confirmed the death but did not cite a specific cause.

Six mornings a week on WMAL (630 AM), using an approach they dubbed “dynamic inaction,” Harden and Weaver reeled off a four-hour roll call of news headlines, weather and traffic reports, school closings, a bit of middle-of-the-road music, some spoofing of the commercials, and visits with a roster of made-up c

In [10]:
import sys, os

In [14]:
sys.path.append(os.getenv('HOME') + '/Packages/MITIE/mitielib')

In [16]:
words = article.text.split(' ')

In [18]:
tokens = mitie.tokenize(article.text)

In [19]:
ner = mitie.named_entity_extractor(os.getenv('HOME') + '/Packages/MITIE/MITIE-models/english/ner_model.dat')

In [20]:
entities = ner.extract_entities(tokens)

In [23]:
print(len(entities))
print(entities)

94
[(range(30, 31), 'LOCATION', 0.8180789172015295), (range(48, 50), 'PERSON', 1.1432234344579681), (range(51, 53), 'PERSON', 0.955392130469865), (range(64, 65), 'LOCATION', 0.8901861857583594), (range(66, 68), 'PERSON', 0.8134518653502155), (range(77, 79), 'LOCATION', 0.45462660011900474), (range(80, 81), 'LOCATION', 0.9274335473066541), (range(95, 96), 'PERSON', 1.1466745285694386), (range(106, 107), 'LOCATION', 0.9919255242968098), (range(114, 115), 'ORGANIZATION', 0.5144205951866204), (range(117, 118), 'LOCATION', 0.3562212502116301), (range(134, 136), 'PERSON', 1.5828000198241443), (range(153, 154), 'ORGANIZATION', 0.32051226227497104), (range(164, 165), 'PERSON', 0.1855682508931393), (range(169, 170), 'PERSON', 0.9745646939543853), (range(171, 172), 'PERSON', 1.040379407150331), (range(309, 311), 'PERSON', 0.9083580090616551), (range(314, 316), 'PERSON', 1.0848929368001634), (range(320, 322), 'PERSON', 1.3668545705457467), (range(330, 333), 'PERSON', 0.6504648022581516), (range(3

In [27]:
entity_texts = []
for e in entities:
    range = e[0]
    tag = e[1]
    score = e[2]
    score_text = "{:0.3f}".format(score)
    entity_text = " ".join(tokens[i].decode() for i in range)
    entity_texts.append(entity_text)
    #print("   Score: " + score_text + ": " + tag + ": " + entity_text)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [29]:
tfidf = 

['D.C.',
 'Frank Harden',
 'Jackson Weaver',
 'Washington',
 'Mr Harden',
 'Chevy Chase',
 'Md',
 'Weaver',
 'Washington',
 'Redskins',
 'Beltway',
 'Robert Harden',
 'WMAL',
 '“',
 'Harden',
 'Weaver',
 'Mr Harden',
 'Jackson Weaver',
 'Frank Harden',
 'Vincent A. Finnigan/Planned',
 'Harden',
 'Weaver',
 'Rocky Rockmont',
 'Rockmont Chevrolet',
 'Rocky .',
 'Mr Harden',
 'Harden',
 'Weaver',
 'Weaver',
 'Mr Harden',
 'Tim Brant',
 'Andy Parks',
 'Bob Daugherty',
 'Harden',
 'Weaver',
 'U.S.',
 'Dr Lucifer Headcold',
 "Albino State Teachers ' College",
 'Symphonic',
 'Boscoe Osgood',
 'Wally Ballou',
 'Bob',
 'Ray',
 'Harden',
 'Weaver',
 'Harden',
 'Weaver',
 'Don Geronimo',
 'Mike Show',
 'Mr Harden',
 'Robert',
 'Washington',
 'Mr Harden',
 'Washington Post',
 'Harden',
 'Weaver',
 'Billy Martin',
 'Carriage House',
 'Georgetown',
 'Mr Harden',
 "Children 's National Medical Center",
 'Harden and Weaver Show',
 'Kennedy Center',
 'Mr Harden',
 'Tom Bresnahan',
 'Bresnahan',
 'WMAL'